# Implementing facial key points detection
This task is considerd as a regression problem, where the prediction isn't a single value, but several continuous outputs. <br>The objective is to detect the key points present on an image of a face.

In [ ]:
import numpy as np
import pandas as pd
import os
import glob
import cv2
from copy import deepcopy

from sklearn import cluster
from sklearn.model_selection import train_test_split

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import matplotlib.ticker as mticker

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision

from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
from torch.utils.data import Dataset

from torchvision import transforms, models, datasets
from torchsummary import summary

from mpl_toolkits.mplot3d import Axes3D

In [ ]:
%matplotlib inline

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

#### Downloading and importing the relevant data. that contains images and their corresponding facial key points.

In [ ]:
!git clone https://github.com/udacity/P1_Facial_Keypoints.git
!cd P1_Facial_Keypoints

In [6]:
root_dir = 'P1_Facial_Keypoints/data/training/'
all_img_paths = glob.glob(os.path.join(root_dir, '*.jpg'))

data = pd.read_csv('P1_Facial_Keypoints/data/training_frames_keypoints.csv')

#### Defining the FacesData class, that provides input and output data points for the data loader

In [7]:
class FacesData(Dataset):
    
    def __init__(self, df):
        
        super(FacesData).__init__()
        self.df = df
        self.normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                              std=[0.229, 0.224, 0.225])
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, ix):
        
        img_path = 'P1_Facial_Keypoints/data/training/' + self.df.iloc[ix,0]
        img = cv2.imread(img_path) / 255.0
        
        kp = deepcopy(self.df.iloc[ix,1:].tolist())
        kp_x = (np.array(kp[0::2])/img.shape[1]).tolist()
        kp_y = (np.array(kp[1::2])/img.shape[0]).tolist()
        kp2 = kp_x + kp_y
        kp2 = torch.tensor(kp2)
        
        img = self.preprocess_input(img)
        
        return img, kp2
    
    def preprocess_input(self, img):
        
        img = cv2.resize(img, (224,224))
        img = torch.tensor(img).permute(2,0,1)
        img = self.normalize(img).float()
        
        return img.to(device)
    
    def load_img(self, ix):
        
        img_path = 'P1_Facial_Keypoints/data/training/' + self.df.iloc[ix,0]        
        img = cv2.imread(img_path)
        img =cv2.cvtColor(img, cv2.COLOR_BGR2RGB)/255.
        img = cv2.resize(img, (224,224))
        
        return img

#### creating a training and test data split, and establish training and test datasets and data loaders

In [8]:
train, test = train_test_split(data, test_size=0.2, random_state=101)

train_dataset = FacesData(train.reset_index(drop=True))
test_dataset = FacesData(test.reset_index(drop=True))

train_loader = DataLoader(train_dataset, batch_size=32)
test_loader = DataLoader(test_dataset, batch_size=32)

#### defining the model, loss function and optimizer

In [9]:
def get_model():
    
    model = models.vgg16(pretrained=True)
    
    for param in model.parameters():
        param.requires_grad = False
        
    model.avgpool = nn.Sequential(nn.Conv2d(512,512,3),
                                  nn.MaxPool2d(2),
                                  nn.Flatten())
    
    model.classifier = nn.Sequential(nn.Linear(2048, 512),
                                     nn.ReLU(),
                                     nn.Dropout(0.5),
                                     nn.Linear(512, 136),
                                     nn.Sigmoid())
    criterion = nn.L1Loss()
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
    
    return model.to(device), criterion, optimizer

In [ ]:
model, criterion, optimizer = get_model()

#### Defining functions to train on a batch of data points and also to validate on the test dataset

In [12]:
def train_batch(img, kps, model, optimizer, criterion):
    
    model.train()
    optimizer.zero_grad()
    _kps = model(img.to(device))
    loss = criterion(_kps, kps.to(device))
    loss.backward()
    optimizer.step()
    
    return loss

#### Defining a function that returns the loss on test data and the predicted key points

In [13]:
@torch.no_grad()
def validate_batch(img, kps, model, criterion):
    
    model.eval()
    _kps = model(img.to(device))
    loss = criterion(_kps, kps.to(device))
    
    return _kps, loss

#### Training the model based on training the data loader and test it on test data

In [ ]:
train_loss, test_loss = [], []
n_epochs = 50

for epoch in range(n_epochs):
    print(f" epoch {epoch+ 1} : 50")
    epoch_train_loss, epoch_test_loss = 0, 0
    
    for ix, (img,kps) in enumerate(train_loader):
        loss = train_batch(img, kps, model, optimizer, criterion)
        epoch_train_loss += loss.item()
        
    epoch_train_loss /= (ix+1)

    for ix,(img,kps) in enumerate(test_loader):
        ps, loss = validate_batch(img, kps, model, criterion)
        epoch_test_loss += loss.item()
        
    epoch_test_loss /= (ix+1)

    train_loss.append(epoch_train_loss)
    test_loss.append(epoch_test_loss)

 epoch 1 : 50


#### Ploting the training and test loss over increasing epochs

In [ ]:
epochs = np.arange(50)+1

plt.plot(epochs, train_loss, 'bo', label='Training loss')
plt.plot(epochs, test_loss, 'r', label='Test loss')

plt.title('Training and Test loss over increasing epochs')

plt.xlabel('Epochs')
plt.ylabel('Loss')

plt.legend()
plt.grid('off')
plt.show()

#### Testing the model on a test image

In [ ]:
ix = 0

plt.figure(figsize=(10,10))
plt.subplot(221)

plt.title('Original image')

im = test_dataset.load_img(ix)
plt.imshow(im)

plt.grid(False)
plt.subplot(222)

plt.title('Image with facial keypoints')

x, _ = test_dataset[ix]
plt.imshow(im)

kp = model(x[None]).flatten().detach().cpu()

plt.scatter(kp[:68]*224, kp[68:]*224, c='r')
plt.grid(False)
plt.show()